# Init

In [1]:
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from pathlib import Path
import json
%cd ..
%load_ext autoreload

c:\Users\kepes\Projects\segmentation


In [2]:
%autoreload 2
from src.MapData import MapData, min_max_dist, triangle_corr, get_triangular_kernel
from src.LineFinder import LineFinder

from src.plotting_functions import plot_embedding, plot_single_variable_map, _update_layout

# Data

In [3]:
map_data = MapData(
    'C:/Users/kepes/OneDrive - Vysoké učení technické v Brně/projects/P42B/2022_03_21_P42B_513x632.libsdata',
    overwrite=False
)
map_data.get_metadata()
map_data.load_wavelenths()
map_data.load_all_data('preprocessed')
map_data.get_map_dimensions()
map_data.trim_spectra(64)
map_data.estimate_systemic_noise()
map_data.get_baseline(
    min_window_size=50,
    smooth_window_size=100
)
map_data.baseline_correct()
map_data.upsample_spectra()
if map_data.systemic_noise_spectrum is not None:
    map_data.denoise_spectra(
        file_name_supplement='preprocessed',
        threshold=np.std(map_data.systemic_noise_spectrum)
    )
maxima_spectrum = map_data.spectra.max(axis=0)

loading metadata
loading wavelengths
getting map dimensions


In [4]:
line_finder = LineFinder(
    maxima_spectrum,
    map_data.wvl,
    name='maxima'
)
line_finder.find_lines()
line_finder.load_nist_tables(    
    Path('C:/Users/kepes/OneDrive - Vysoké učení technické v Brně/projects/marsData/inventory/nistTables')
)
line_finder.find_peaks_in_reference(
    maxima_spectrum, 
    scale=False,
    show_cond=False
)
line_finder.plot.show()

creating base plot


## Line intensities

### Denoised

In [5]:
map_data.set_emission_line_parameters(
    line_finder.peaks[1].get('left_bases'),
    line_finder.peaks[1].get('right_bases'),
    line_centers=line_finder.peaks[0]
)
map_data.set_emisssion_line_functions(    
    intensity_funcs=[np.max,np.sum,min_max_dist,triangle_corr]
)
map_data.get_emission_line_intensities()

loading emission line intensities
converting line intensities to arrays


In [6]:
line_centers = list(map_data.line_intensities['amax'].keys())
for ndx,line_center in enumerate(line_centers):
    print(f'{ndx} :: {line_center}')

0 :: 247.96
1 :: 249.28
2 :: 263.27
3 :: 266.12
4 :: 268.25
5 :: 279.63
6 :: 280.35
7 :: 285.30
8 :: 316.04
9 :: 318.05
10 :: 341.57
11 :: 387.07
12 :: 388.23
13 :: 393.34
14 :: 396.83


In [9]:
for line_ndx in [0,5,13]:
    line_center = line_centers[line_ndx]
    for func in map_data.line_intensities:
        plot_single_variable_map(
            map_data.vector_to_array(map_data.line_intensities[func][line_center]),
            figure_title=f'{func}({line_center} nm)',
            fig_size_scaling=2,
            file_id=f"denoisingImpactDenoised_{func}_{line_center.replace('.','_')}nm"
        )
        plt.close()

c:\Users\kepes\Projects\segmentation\src\plotting_functions.py:192: UserWarning:

Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.



### Raw

In [14]:
map_data = MapData(
    'C:/Users/kepes/OneDrive - Vysoké učení technické v Brně/projects/P42B/2022_03_21_P42B_513x632.libsdata',
    overwrite=False
)
map_data.get_metadata()
map_data.load_wavelenths()
map_data.load_all_data('trial')
map_data.get_map_dimensions()
map_data.trim_spectra(64)
map_data.get_baseline(
    min_window_size=50,
    smooth_window_size=100
)
map_data.baseline_correct()
map_data.upsample_spectra()

map_data.set_emission_line_parameters(
    line_finder.peaks[1].get('left_bases'),
    line_finder.peaks[1].get('right_bases'),
    line_centers=line_finder.peaks[0]
)
map_data.set_emisssion_line_functions(    
    intensity_funcs=[np.max,np.sum,min_max_dist,triangle_corr]
)
map_data.get_emission_line_intensities(overwrite=True)

loading metadata
loading wavelengths
preprocessed file was not found; setting overwrite to True
loading raw data
getting map dimensions
getting baselines
upsampling spectra
extracting emission line intensities using amax
extracting emission line intensities using sum
extracting emission line intensities using min_max_dist
extracting emission line intensities using triangle_corr
converting line intensities to lists
saving emission line intensities


In [15]:
map_data.get_emission_line_intensities(overwrite=False)
for line_ndx in [0,5,13]:
    line_center = line_centers[line_ndx]
    for func in map_data.line_intensities:
        plot_single_variable_map(
            map_data.vector_to_array(map_data.line_intensities[func][line_center]),
            figure_title=f'{func}({line_center} nm)',
            fig_size_scaling=2,
            file_id=f"denoisingImpactBaselined_{func}_{line_center.replace('.','_')}nm"
        )
        plt.close()

loading emission line intensities
converting line intensities to arrays


c:\Users\kepes\Projects\segmentation\src\plotting_functions.py:192: UserWarning:

Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.

